In [1]:
import pandas as pd

Este notebook es para unir los resultados de los 3 mecanismos alternativos de selección:

- CNO 3 digitos
- IPO no ponderado
- Transformación convexa 

In [3]:
def group_data(data, keys = ["Ruta habilitada", "Ocupación"]):
    resumen = data.groupby(
        by=keys,
        dropna=False
    ).agg(
        IPO=("IPO", "mean"),
        cupos_habilitados=("cupos_habilitados", "sum"),
        monto_asignado = ("monto_asignado","sum")
    ).reset_index()
    return resumen

In [10]:
# Función para renombrar columnas con sufijo, excepto las claves
def renombrar_columnas(df, sufijo, claves):
    return df.rename(columns={
        col: f"{col}_{sufijo}" for col in df.columns if col not in claves
    })


In [11]:
pd.set_option('display.float_format', '{:,.2f}'.format)

In [13]:
# Cargar archivos
cno3 = pd.read_excel("Export/CNO3/resumen_via_cno3.xlsx")
ipo_no_ponderado = pd.read_excel("Export/SinPonderar/resumen_via_sinponderar.xlsx")
convexa = pd.read_excel("Export/Convexa/resumen_via_convexa.xlsx")
original = pd.read_excel("Export/resumen_via_original.xlsx")

cno3 = cno3.drop(columns=["Unnamed: 0"], errors='ignore')
ipo_no_ponderado = ipo_no_ponderado.drop(columns=["Unnamed: 0"], errors='ignore')
convexa = convexa.drop(columns=["Unnamed: 0"], errors='ignore')
original = original.drop(columns=["Unnamed: 0"], errors='ignore')

In [14]:
cno3_grouped = group_data(cno3)
ipo_no_ponderado_grouped = group_data(ipo_no_ponderado)
convexa_grouped = group_data(convexa)
original_grouped = group_data(original)

In [15]:
# Llaves de unión (sin 'nombre_institucion')
keys = ["Ruta habilitada", "Ocupación"]

# Renombrar columnas
original_grouped = renombrar_columnas(original_grouped, "original", keys)
cno3_grouped = renombrar_columnas(cno3_grouped, "cno3", keys)
ipo_no_ponderado_grouped = renombrar_columnas(ipo_no_ponderado_grouped, "ipo", keys)
convexa_grouped = renombrar_columnas(convexa_grouped, "convexa", keys)

# Hacer merge progresivamente
df_merge = original_grouped.merge(cno3_grouped, on=keys, how='outer')
df_merge = df_merge.merge(ipo_no_ponderado_grouped, on=keys, how='outer')
df_final = df_merge.merge(convexa_grouped, on=keys, how='outer')

# Ordenar
df_final = df_final.sort_values(
    by=["Ruta habilitada","monto_asignado_original", "monto_asignado_cno3", "monto_asignado_ipo", "monto_asignado_convexa"],
    ascending=[True,False, False, False, False]
)

# Quitar decimales
cols = ["monto_asignado_original","monto_asignado_convexa", "monto_asignado_ipo", "monto_asignado_cno3"]
df_final[cols] = df_final[cols].fillna(0).astype(int)

df_final = df_final.reset_index(drop=True)

In [16]:
df_final.to_excel("asignaciones_por_ocupacion_mecanismos_alternativos.xlsx")

In [19]:
df_final[['Ruta habilitada','monto_asignado_original','monto_asignado_cno3','monto_asignado_ipo','monto_asignado_convexa']]

,Ruta habilitada,monto_asignado_original,monto_asignado_cno3,monto_asignado_ipo,monto_asignado_convexa
0,Antiguos,227058468,227058468,97310772,210840006
1,Antiguos,215523078,215523078,109528121,219056243
2,Antiguos,173176347,173176347,110640444,177986801
3,Antiguos,153661017,153661017,87806295,131709443
4,Antiguos,110572617,110572617,123581161,123581161
5,Antiguos,106624039,106624039,95593966,95593966
6,Antiguos,106447062,159670593,119752945,113100003
7,Antiguos,86588173,86588173,173176347,120261352
8,Antiguos,49340892,0,88813606,39472713
9,Antiguos,42623849,42623849,106559624,42623849
